In [ ]:
pip install pandas openpyxl sastrawi nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import string

In [ ]:
# Download NLTK data
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
# Download NLTK data
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Baca data dari file .xlsx
df = pd.read_excel('/content/Turnbackhoax.xlsx')  # ganti dengan nama file kamu

# Cek kolom
print(df.columns)

Index(['Title', 'Timestamp', 'FullText', 'CleanNarasi', 'Url', 'label',
       'sumber', 'scrap'],
      dtype='object')


In [ ]:
# Cek tipe data unik di kolom CleanNarasi
print("Tipe data unik di CleanNarasi:")
print(df['CleanNarasi'].apply(type).value_counts())

# Cek apakah ada nilai NaN (meskipun seharusnya sudah diisi)
print("\nJumlah NaN di CleanNarasi:")
print(df['CleanNarasi'].isna().sum())

# Cek apakah ada nilai None
print("\nJumlah None di CleanNarasi:")
print(df['CleanNarasi'].isna().sum())  # None juga akan terdeteksi sebagai NaN

# Cek sample data
print("\nSample data CleanNarasi:")
print(df['CleanNarasi'].head(10))

Tipe data unik di CleanNarasi:
CleanNarasi
<class 'str'>      6328
<class 'float'>    4051
Name: count, dtype: int64

Jumlah NaN di CleanNarasi:
4051

Jumlah None di CleanNarasi:
4051

Sample data CleanNarasi:
0    BISA DILIHAT SI ONTA YAMAN NGGAK PEDULI ITU AP...
1    ini bener gasih?? Ya Allah gimna keadaan pa ha...
2    Nenek lampir pemimpin partai banteng bercula s...
3    gerombolan kulup banyak menyusup ke ormas2 isl...
4                                                  NaN
5    M g w ti Di Panggil B4waslu:exclamation: Bvntu...
6    INNALILLAHI: Jenajah Tiba di Rumah Duka Ari La...
7    Mengejutkan || Bharada E dan Ronny Hadiri Pema...
8    : Ini adalah foto yang paling menyedihkan. Zel...
9    : Sementara itu, di Arab Saudi Salju turun unt...
Name: CleanNarasi, dtype: object


In [ ]:
# Ganti NaN dan None dengan string kosong, lalu ubah semua ke string
df['CleanNarasi'] = df['CleanNarasi'].fillna("").astype(str)

In [ ]:
# Hitung total baris yang kosong atau hanya berisi spasi
total_kosong = df[df['CleanNarasi'].str.strip() == ''].shape[0]
print(f"Total baris Clean Narasi kosong atau hanya spasi: {total_kosong}")

Total baris Clean Narasi kosong atau hanya spasi: 4051


In [ ]:
df['CleanedNarasi'] = df['CleanNarasi'].str.lower()

In [ ]:
def clean_text(text):
    # Hilangkan emoji
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Hilangkan simbol kecuali huruf, angka, spasi
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    # Hilangkan tanda baca berlebihan
    text = re.sub(r'[^\w\s]', ' ', text)
    return text

df['CleanedNarasi'] = df['CleanedNarasi'].apply(clean_text)

In [ ]:
df['count_exclamation'] = df['CleanNarasi'].apply(lambda x: x.count('!'))
df['count_question'] = df['CleanNarasi'].apply(lambda x: x.count('?'))

In [ ]:
df['tokens'] = df['CleanedNarasi'].apply(word_tokenize)

In [ ]:
# Kamus normalisasi dari file "combined_slang_words"
# Asumsi file ini adalah CSV atau teks dengan delimiter yang sesuai,
# dan memiliki kolom kata tidak baku dan kata baku.

try:
    # Coba baca file dengan delimiter koma (sesuaikan jika delimiter berbeda)
    slang_df = pd.read_csv('/content/combined_slang_words.txt', sep=',', header=None)
    # Buat dictionary dari DataFrame
    normalization_dict = dict(zip(slang_df[0], slang_df[1]))
except Exception as e:
    print(f"Error reading combined_slang_words.txt: {e}")
    print("Using a small placeholder dictionary instead.")
    # Gunakan kamus placeholder jika gagal membaca file
    normalization_dict = {
        'yg': 'yang',
        'gak': 'tidak',
        'ga': 'tidak',
        'bikin': 'membuat',
        'ngecek': 'mengecek',
        'gimna': 'bagaimana',
        'gasih': 'tidak',
        'pa': 'pak',
        'krstn': 'kristen',
        'gasih' : 'ga',
        'kang' : 'tukang'
    }


def normalize_indonesian(tokens, normalization_dict):
    # Pastikan tokens adalah list of strings
    if not isinstance(tokens, list):
        return [] # Return empty list if input is not a list

    return [normalization_dict.get(word, word) for word in tokens]

# Terapkan normalisasi bahasa baku setelah penghapusan stopword
# Menggunakan kolom 'tokens_no_stopwords' sebagai input
df['normalized_tokens'] = df['tokens'].apply(lambda tokens: normalize_indonesian(tokens, normalization_dict))

# Anda bisa cek hasilnya
display(df[['tokens', 'normalized_tokens']].head())

,tokens_no_stopwords,normalized_tokens
0,"[dilihat, si, onta, yaman, peduli, apa, pentin...","[dilihat, si, onta, yaman, peduli, apa, pentin..."
1,"[bener, gasih, allah, gimna, keadaan, pa, haki...","[bener, gasih, allah, gimna, keadaan, pa, haki..."
2,"[nenek, lampir, pemimpin, partai, banteng, ber...","[nenek, lampir, pemimpin, partai, banteng, ber..."
3,"[gerombolan, kulup, banyak, menyusup, ormas, i...","[gerombolan, kulup, banyak, menyusup, ormas, i..."
4,[],[]


In [ ]:
# Penghapusan Stopword
factory = StopWordRemoverFactory()
stopwords_remover = factory.create_stop_word_remover()

df['tokens_no_stopwords'] = df['normalized_tokens'].apply(lambda tokens: stopwords_remover.remove(" ".join(tokens)).split())

In [ ]:
def extract_features(text):
    # Jumlah huruf kapital beruntun
    capital_seq = len(re.findall(r'[A-Z]{2,}', text))
    # Jumlah tanda seru berturut-turut
    exclamation_seq = len(re.findall(r'!{2,}', text))
    # Panjang kalimat dalam jumlah kata
    word_count = len(text.split())
    # Kata hiperbolis
    hyperbolic_words = ["heboh", "bikin geger", "mengerikan", "tak terduga"]
    hyperbolic_count = sum([1 for word in hyperbolic_words if word in text])

    return pd.Series([capital_seq, exclamation_seq, word_count, hyperbolic_count])

df[['capital_seq', 'exclamation_seq', 'word_count', 'hyperbolic_count']] = df['CleanNarasi'].apply(extract_features)

In [ ]:
df[['CleanNarasi', 'CleanedNarasi', 'count_exclamation', 'count_question',
    'capital_seq', 'exclamation_seq', 'word_count', 'hyperbolic_count', 'tokens', 'tokens_no_stopwords', 'normalized_tokens']].head()

,CleanNarasi,CleanedNarasi,count_exclamation,count_question,capital_seq,exclamation_seq,word_count,hyperbolic_count,tokens,tokens_no_stopwords,normalized_tokens
0,BISA DILIHAT SI ONTA YAMAN NGGAK PEDULI ITU AP...,bisa dilihat si onta yaman nggak peduli itu ap...,0,0,17,0,17,0,"[bisa, dilihat, si, onta, yaman, nggak, peduli...","[dilihat, si, onta, yaman, peduli, apa, pentin...","[dilihat, si, onta, yaman, peduli, apa, pentin..."
1,ini bener gasih?? Ya Allah gimna keadaan pa ha...,ini bener gasih ya allah gimna keadaan pa ha...,0,3,0,0,19,0,"[ini, bener, gasih, ya, allah, gimna, keadaan,...","[bener, gasih, allah, gimna, keadaan, pa, haki...","[bener, gasih, allah, gimna, keadaan, pa, haki..."
2,Nenek lampir pemimpin partai banteng bercula s...,nenek lampir pemimpin partai banteng bercula s...,0,0,0,0,44,0,"[nenek, lampir, pemimpin, partai, banteng, ber...","[nenek, lampir, pemimpin, partai, banteng, ber...","[nenek, lampir, pemimpin, partai, banteng, ber..."
3,gerombolan kulup banyak menyusup ke ormas2 isl...,gerombolan kulup banyak menyusup ke ormas isl...,0,0,2,0,44,0,"[gerombolan, kulup, banyak, menyusup, ke, orma...","[gerombolan, kulup, banyak, menyusup, ormas, i...","[gerombolan, kulup, banyak, menyusup, ormas, i..."
4,,,0,0,0,0,0,0,[],[],[]


In [ ]:
df.to_excel('turnbackhoax_preprocessed.xlsx', index=False)